<a href="https://colab.research.google.com/github/gary-butler/NEAT/blob/master/NEAT_Forex_01_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##################################################

!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

!pip install gym
!pip install pandas

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
python-dev is already the newest version (2.7.15~rc1-1).
python-numpy is already the newest version (1:1.13.3-2ubuntu1).
python-numpy set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
The following additional packages will be installed:
  gir1.2-ibus-1.0 libcapnp-0.6.1 libdbus-1-dev libdmx-dev libdmx1
  libfontenc-dev libfs-dev libfs6 libibus-1.0-5 libibus-1.0-dev
  libmirclient-dev libmirclient9 libmircommon-dev libmircommon7
  libmircookie-dev libmircookie2 libmircore-dev libmircore1 libmirprotobuf3
  libpciaccess-dev libpixman-1-dev libprotobuf-dev libprotobuf-lite10
  libpulse-dev libpu

In [0]:
##################################################

from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
##################################################

import math
import random
import pandas as pd
import numpy as np
import gym
from gym import spaces
#from stable_baselines.common.vec_env import DummyVecEnv

##################################################

_TickTime   = 0
_BidPrice   = 1
_AskPrice   = 2

_H1_Start   = 3
_H1_Action  = 4
_H1_Alpha_0 = 5
_H1_Alpha_1 = 6
_H1_Alpha_2 = 7
_H1_Alpha_3 = 8
_H1_Alpha_4 = 9
_H1_Beta_0  = 10
_H1_Beta_1  = 11
_H1_Beta_2  = 12
_H1_Beta_3  = 13
_H1_Beta_4  = 14

_H4_Start   = 15
_H4_Action  = 16
_H4_Alpha_0 = 17
_H4_Alpha_1 = 18
_H4_Alpha_2 = 19
_H4_Alpha_3 = 20
_H4_Alpha_4 = 21
_H4_Beta_0  = 22
_H4_Beta_1  = 23
_H4_Beta_2  = 24
_H4_Beta_3  = 25
_H4_Beta_4  = 26

#filename = 'Gym01-TrainingData-GBPUSDe.csv'
filename = '/content/drive/My Drive/Gym01-TrainingData-GBPUSDe.csv'

df = pd.read_csv(
    filename,
    sep=';',
    header=None,
)

In [0]:
##################################################

CONFIG_INPUT_SIZE = 20
CONFIG_GENERATION_SIZE = 1000
CONFIG_POPULATION_SIZE = 50
CONFIG_TRAINING_STEPS = 10000
CONFIG_MUTATION_RATE = 0.01

CONFIG_ACCOUNT_BALANCE = 1000
CONFIG_ACCOUNT_LEVERAGE = 200

CONFIG_ORDER_SYMBOL = 'GBPUSDe'
CONFIG_ORDER_BASE_USD = False
CONFIG_ORDER_POINT_SIZE = 0.00001
CONFIG_ORDER_CONTRACT_SIZE = 100000

CONFIG_ORDER_MIN_MARGIN_LEVEL = 2
CONFIG_ORDER_LOTS_PER_1000_USD = 0.3
CONFIG_ORDER_TAKE_PROFIT = 100
CONFIG_ORDER_STOP_LOSS = -50
CONFIG_ORDER_COMMISION = -6
CONFIG_ORDER_STOP_OUT = 0.5

CONFIG_ORDER_MARGIN = CONFIG_ORDER_CONTRACT_SIZE / CONFIG_ACCOUNT_LEVERAGE
CONFIG_ORDER_POINT_PROFIT = CONFIG_ORDER_CONTRACT_SIZE * CONFIG_ORDER_POINT_SIZE

##################################################

ORDER_TYPE_NONE = 0
ORDER_TYPE_BUY = 1
ORDER_TYPE_SELL = 2

ORDER_CLOSE_NONE = 0
ORDER_CLOSE_TAKE_PROFIT = 1
ORDER_CLOSE_STOP_LOSS = 2
ORDER_CLOSE_STOP_OUT = 3

class ForexOrder:
    def __init__(
        self,
        _symbol,
        _type,
        _lots,
        _open_time,
        _open_price,
        _take_profit,
        _stop_loss,
        _commision,
        _margin
    ):
        self.symbol = _symbol
        self.type = _type
        self.lots = _lots
        self.open_time = _open_time
        self.open_price = _open_price
        self.take_profit = _take_profit
        self.stop_loss = _stop_loss
        self.commision = _commision
        self.margin = _margin
        
        self.duration = 0
        self.point_profit = 0
        self.profit_points = 0
        self.profit_usd = 0
        
        self.close = ORDER_CLOSE_NONE
        
        self._point_profit = 0
            
        if CONFIG_ORDER_BASE_USD:
            self._point_profit = CONFIG_ORDER_POINT_PROFIT * self.lots / CONFIG_ORDER_POINT_SIZE
        else:
            self._point_profit = CONFIG_ORDER_POINT_PROFIT * self.lots
        
    def summary(self):
        print('    ========== ForexOrder.summary ==========')
        print('    - symbol = ', self.symbol)
        print('    - type = ', self.type)
        print('    - lots = ', self.lots)
        print('    - open_time = ', self.open_time)
        print('    - open_price = ', self.open_price)
        print('    - take_profit = ', self.take_profit)
        print('    - stop_loss = ', self.stop_loss)
        print('    - commision = ', self.commision)
        print('    - margin = ', self.margin)
        print('')
        print('    - duration = ', self.duration)
        print('    - point_profit = ', self.point_profit)
        print('    - profit_points = ', self.profit_points)
        print('    - profit_usd = ', self.profit_usd)
        print('')
        print('    - close = ', self.close)
        

##################################################

BAR_TYPE_NONE = 0
BAR_TYPE_UP = 1
BAR_TYPE_DOWN = 2

TREND_TYPE_NONE = 0
TREND_TYPE_UP = 1
TREND_TYPE_DOWN = 2

class ForexEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    
    def __init__(self, df):
        super(ForexEnv, self).__init__()
        
        self.df = df
        self.df_size = len(df.index)
        
    def reset(self, generation):
        self.balance = CONFIG_ACCOUNT_BALANCE
        self.equity = CONFIG_ACCOUNT_BALANCE
        self.margin = 0
        self.margin_level = 0
        
        self.training_steps = CONFIG_TRAINING_STEPS * (generation + 1)
        self.current_step = random.randint(0, self.df_size - self.training_steps)
        self.stop_step = self.current_step + self.training_steps
        self.stop = ORDER_CLOSE_NONE
        
        self.orders_total = 0
        self.orders = []
        self.orders_start = []
        
        self.tick_time = 0
        self.bid_price = 0
        self.ask_price = 0
        
        self.h1_start = 0
        self.h1_action = 0
        
    def summary(self):
        print('========== ForexEnv.summary ==========')
        print('- balance = ', self.balance)
        print('- equity = ', self.equity)
        print('- margin = ', self.margin)
        print('- margin_level = ', self.margin_level)
        print('')
        print('- current_step = ', self.current_step)
        print('- stop_step = ', self.stop_step)
        print('- stop = ', self.stop)
        print('')
        print('- orders_total = ', self.orders_total)
        print('- orders: ')
        
        for order in self.orders:
            order.summary()
        
    def get_current_observation(self):
        current_observation = np.array([
            self.df.loc[self.current_step, _H1_Alpha_0],
            self.df.loc[self.current_step, _H1_Alpha_1],
            self.df.loc[self.current_step, _H1_Alpha_2],
            self.df.loc[self.current_step, _H1_Alpha_3],
            self.df.loc[self.current_step, _H1_Alpha_4],
            
            self.df.loc[self.current_step, _H1_Beta_0],
            self.df.loc[self.current_step, _H1_Beta_1],
            self.df.loc[self.current_step, _H1_Beta_2],
            self.df.loc[self.current_step, _H1_Beta_3],
            self.df.loc[self.current_step, _H1_Beta_4],
            
            self.df.loc[self.current_step, _H4_Alpha_0],
            self.df.loc[self.current_step, _H4_Alpha_1],
            self.df.loc[self.current_step, _H4_Alpha_2],
            self.df.loc[self.current_step, _H4_Alpha_3],
            self.df.loc[self.current_step, _H4_Alpha_4],
            
            self.df.loc[self.current_step, _H4_Beta_0],
            self.df.loc[self.current_step, _H4_Beta_1],
            self.df.loc[self.current_step, _H4_Beta_2],
            self.df.loc[self.current_step, _H4_Beta_3],
            self.df.loc[self.current_step, _H4_Beta_4],
        ])
        
        return current_observation
    
    def _open_order(self, order_type):
        order_symbol = CONFIG_ORDER_SYMBOL
            
        order_lots = self.equity * CONFIG_ORDER_LOTS_PER_1000_USD / 1000
        order_lots = round(order_lots, 2)
            
        order_open_time = self.tick_time
        order_open_price = 0
        
        if order_type == ORDER_TYPE_BUY:
            order_open_price = self.ask_price
        if order_type == ORDER_TYPE_SELL:
            order_open_price = self.bid_price
            
        order_take_profit = CONFIG_ORDER_TAKE_PROFIT
        order_stop_loss = CONFIG_ORDER_STOP_LOSS
        order_commision = CONFIG_ORDER_COMMISION
        
        order_margin = 0
            
        if CONFIG_ORDER_BASE_USD:
            order_margin = CONFIG_ORDER_MARGIN * order_lots
        else:
            order_margin = CONFIG_ORDER_MARGIN * order_lots * self.bid_price * CONFIG_ORDER_POINT_SIZE
            
        order_margin = round(order_margin, 3)
        
        self.margin += order_margin
          
        order = ForexOrder(
            order_symbol,
            order_type,
            order_lots,
            order_open_time,
            order_open_price,
            order_take_profit,
            order_stop_loss,
            order_commision,
            order_margin
        )
        
        self.orders_total += 1
        self.orders.append(order)
        self.orders_start.append(self.h1_start)
        
        #print('xxxxxxxxxxxxxxxxxxxx _open_order xxxxxxxxxxxxxxxxxxxx')
        #self.summary()
        
    def _close_order(self, order, order_close):
        order.close = order_close
        
        self.balance += order.profit_usd
        self.margin -= order.margin
        
        self.stop = order_close
        
        self.orders_total -= 1
        
        #print('xxxxxxxxxxxxxxxxxxxx _close_order xxxxxxxxxxxxxxxxxxxx')
        #self.summary()
    
    def _calculate_orders(self):
        self.equity = self.balance
        
        for order in self.orders:
            if order.close != ORDER_CLOSE_NONE:
                continue
                
            order.duration = self.tick_time - order.open_time
            
            if CONFIG_ORDER_BASE_USD:
                order.point_profit = order._point_profit / self.ask_price
            else:
                order.point_profit = order._point_profit
                
            order.point_profit = round(order.point_profit, 3)
            
            if order.type == ORDER_TYPE_BUY:
                order.profit_points = self.bid_price - order.open_price
            elif order.type == ORDER_TYPE_SELL:
                order.profit_points = order.open_price - self.ask_price
                
            order.profit_usd = (order.profit_points + order.commision) * order.point_profit
            order.profit_usd = round(order.profit_usd, 3)
                
            if (order.profit_points >= order.take_profit):
                self._close_order(order, ORDER_CLOSE_TAKE_PROFIT)
                continue
                
            if (order.profit_points <= order.stop_loss):
                self._close_order(order, ORDER_CLOSE_STOP_LOSS)
                continue
            
            self.equity += order.profit_usd
            
        self.equity = round(self.equity, 3)
        
        if self.margin:
            self.margin_level = round(self.equity / self.margin, 2)
            if self.margin_level < CONFIG_ORDER_STOP_OUT:
                self.stop = ORDER_CLOSE_STOP_OUT
        
    def _take_action(self, action):
        order_type = action
        
        if order_type != ORDER_TYPE_BUY and order_type != ORDER_TYPE_SELL:
            return
        
        if self.margin_level > 0 and self.margin_level < CONFIG_ORDER_MIN_MARGIN_LEVEL:
            return
          
        self._open_order(order_type)
    
    def should_get_action(self):
        self.tick_time = self.df.loc[self.current_step, _TickTime]
        self.bid_price = self.df.loc[self.current_step, _BidPrice]
        self.ask_price = self.df.loc[self.current_step, _AskPrice]
        
        self.h1_start = self.df.loc[self.current_step, _H1_Start]
        self.h1_action = self.df.loc[self.current_step, _H1_Action]
        
        if self.h1_start in self.orders_start:
            return False
        
        return self.h1_action
    
    def sub_step(self):
        self._calculate_orders()
        
        self.current_step += 1

    def step(self, action):
        self._take_action(action)
        self._calculate_orders()
        
        self.current_step += 1
        
    def is_done(self):
        done = False
        reward = self.equity
        
        if self.current_step >= self.stop_step:
            done = True
            
        if self.stop == ORDER_CLOSE_STOP_OUT:
            done = True
            
        #if done:
        #    self.summary()
            
        return done, reward

In [0]:
#from env.forex_env import ForexEnv

#make df from 'GBPUSDe.csv'

#initialize Forex environment
env = ForexEnv(df)

#observation will be used to determine the action after passing through the neural network
observation = env.reset(0)
#print('line 20: observation:', observation.shape)

#the choice between mates with identical fitness will be random
random.seed()

#the genomes will all be stored in the dataframe in the form of {keys, fitness, nodes, connects}
genomes = pd.DataFrame()
genomes_old = pd.DataFrame()

#each node in nodes will have [h_mark, enabled, weight, bias]
#where h_mark is the historical marker created when a new node is created
#and shared by all nodes decended from the same node
#this makes mating and structual comparison much easier
#enabled is a 1 for an enabled node, 0 for a disabled node
#weight and bias are used as is standard for neural nets

#each connect in connects will have [h_mark, enabled, weight, bias, in_node, out_node]
#where h_mark, enabled, weight, and bias are exactly the same as with nodes
#in_node and out_node are the nodes which are connected by this connect

num_inputs = CONFIG_INPUT_SIZE

#number of generations to run for if we don't find a solution sooner
generations = CONFIG_GENERATION_SIZE

#hist_marker is the cornerstone of NEAT. It increments with each new node or connection 
#Allows comparison accross genes for mating and species division, by giving each
#node and connection a unique marker
hist_marker = 0

#gene_key is a unique ID for each genome, used for mating, fitness testing, mutation, and management
gene_key = 0

#hyperparamiters for neuroevolution of nodes and connections
weight_change_freq = 0.8
weight_uniform_cf = 0.9

#hyperparamiters for compatibility coefficients
c1 = 1.0
c2 = 1.0
c3 = 0.4
delta = 3.0

#generations allowed without improving fitness, if this number is exceded the 
#species is removed from reproduction pool
fitness_no_improve_limit = 15

#the champion of each species with more than this many networks is copied unchanged next generation
champion_copy_minimum = 5

#75% chance that gene disabled if disabled in either parent
disabled_inheritance = 0.75

#25% chance of offspring from pure mutation without crossover
mutate_vs_mate = 0.25

#interspecies mating rate 0.001
intermate_rate = 0.001

#new node and link addition probability
new_node_prob = 0.03
new_link_prob = 0.05

def distance_function(excess_gene_total, disjoint_gene_total, weight_dif_average, greater_gene_count):
    """
    compatability distance function, calculated based on gene differences rather than topolgy
    distance_function(total number of excess genes,
    total number of disjoint genes,
    average weight difference,
    the total number of genes in the larger of the two genomes)
    uses the c1, c2, and c3 compatablity coefficient hyperparamiters to sum up
    the effective "distance" between two geneomes
    in the original NEAT they state the settings for these hyperparamiters and
    to set gene counts of less than 20 egual to 1, which is done automatically here
    """
    global c1
    global c2
    global c3
    if(greater_gene_count < 20):
        greater_gene_count = 1
    arg1 = (c1 * excess_gene_total) / greater_gene_count
    arg2 = (c2 * disjoint_gene_total) / greater_gene_count
    arg3 = c3 * weight_dif_average
    return arg1 + arg2 + arg3

def compatability(a_nodes = [], a_connects = [], b_nodes = [], b_connects = []):
    """
    check if two genomes belong to the same species
    compatability([nodes from first genome],
    [connections from first genome],
    [nodes from second genome],
    [connections from second genome])
    """
    global delta
    E = num_excess_genes(a_nodes, a_connects, b_nodes, b_connects)
    D = num_disjoint_genes(a_nodes, a_connects, b_nodes, b_connects)
    W = average_weight_dif(a_nodes, a_connects, b_nodes, b_connects)
    N = max_num_genes(a_nodes, a_connects, b_nodes, b_connects)
    distance = distance_function(E, D, W, N)
    return (distance < delta)

def relu(z):
    """
    standard ReLu transfer/activation function
    returns z or 0 if z < 0
    included for testing
    """
    if(z < 0):
        z = 0
    return z
         
    
def sigmoid(z):
    """
    modified sigmoidal transfer function suggested by the original NEAT
    """
    return 1 / (1 + math.e ^ (-4.9 * z))

#this was for the Cartpole OpenAI Gym problem, not Forex NEAT
#the get_action function results in a 4 centered float, we need an action of 0 or 1
#this could be improved, but until then this binary_activation gets the job done
def trinary_activation(z):
    """
    the get_action() function results in a 4.0 centered float, we need an action of 0 or 1
    this could be improved, but until then this binary_activation gets the job done
    this issue is not addressed in the original NEAT
    """
    if(z > 2):
        return 2
    elif(z > 1):
        return 1
    return 0

def num_excess_genes(a_nodes = [], a_connects = [], b_nodes = [], b_connects = []):
    """
    excess genes are defined as the extra genes found in the middle sections of two
    compared genomes, used for calculating the distance between the two genomes
    first we find the highest matching gene, then we sum the non matching genes
    between the beginning of each genome and the highest matching gene
    num
    """
    num_e = 0
    highest_match = 0
    for a_node in a_nodes:
        for b_node in b_nodes:
            if (a_node[0] == b_node[0]):
                highest_match = max(highest_match, a_node[0])
    for a_connect in a_connects:
        for b_connect in b_connects:
            if (a_connect[0] == b_connect[0]):
                highest_match = max(highest_match, a_connect[0])
    for a_node in a_nodes:
        if(a_node[0] > highest_match):
            num_e += 1
    for b_node in b_nodes:    
        if(b_node[0] > highest_match):
            num_e += 1
    for a_connect in a_connects:
        if(a_connect[0] > highest_match):
            num_e += 1
    for b_connect in b_connects:
        if(b_connect[0] > highest_match):
            num_e += 1
    return num_e

def num_disjoint_genes(a_nodes = [], a_connects = [], b_nodes = [], b_connects = []):
    num_d = 0
    highest_match = 0
    for a_node in a_nodes:
        for b_node in b_nodes:
            if (a_node[0] == b_node[0]):
                highest_match = max(highest_match, a_node[0])
    for a_connect in a_connects:
        for b_connect in b_connects:
            if (a_connect[0] == b_connect[0]):
                highest_match = max(highest_match, a_connect[0])
    for a_node in a_nodes:
        excess = True
        for b_node in b_nodes:
            if (a_node[0] == b_node[0]):
                excess = False
        if((excess) and (a_node[0] < highest_match)):
            num_d += 1
    for a_connect in a_connects:
        excess = True
        for b_connect in b_connects:
            if (a_connect[0] == b_connect[0]):
                excess = False
        if((excess) and (a_node[0] < highest_match)):
            num_d += 1
    for b_node in b_nodes:
        excess = True
        for a_node in a_nodes:
            if (a_node[0] == b_node[0]):
                excess = False
        if((excess) and (b_node[0] < highest_match)):
            num_d += 1
    for b_connect in b_connects:
        excess = True
        for a_connect in a_connects:
            if (a_connect[0] == b_connect[0]):
                excess = False
        if((excess) and (b_node[0] < highest_match)):
            num_d += 1
    return num_d

def average_weight_dif(a_nodes = [], a_connects = [], b_nodes = [], b_connects = []):
    match_count = 0
    weight_dif_sum = 0
    bias_dif_sum = 0
    for a_node in a_nodes:
        for b_node in b_nodes:
            if(a_node[0] == b_node[0]):
                match_count += 1
                weight_dif_sum += b_node[2] - a_node[2]
                bias_dif_sum = b_node[3] - a_node[3]
    for a_connect in a_connects:
        for b_connect in b_connects:
            if(a_connect[0] == b_connect[0]):
                match_count += 1
                weight_dif_sum += b_connect[2] - a_connect[2]
                bias_dif_sum = b_connect[3] - a_connect[3]
    ave_dif = (weight_dif_sum + bias_dif_sum)/ (2 * match_count)
    return ave_dif

def max_num_genes(a_nodes = [], a_connects = [], b_nodes = [], b_connects = []):
    return max(len(a_nodes) + len(a_connects), len(b_nodes) + len(b_connects))

#creates a new connection between in_node and out_node, initializes the weight and bias
#defaults enabled to 1(True) grants it a unique hist_marker and increments the hist_marker
#then it returns the set as a python list
def new_connect(in_node, out_node, init = False, i_ = 0):
    weight = 1
    bias = 1
    enabled = 1
    global hist_marker
    h_mark = hist_marker
    if(init and (hist_marker < 1 + num_inputs)):
        hist_marker += 1
    elif(init):
        h_mark = i_
    elif(not init):
        hist_marker += 1
    return [h_mark, enabled, weight, bias, in_node, out_node]

def new_node(init = False, i_ = 0):
    bias = 1
    weight = 1
    enabled = 1
    global hist_marker
    h_mark = hist_marker
    if(init and (hist_marker < 2 + num_inputs)):
        hist_marker += 1
    elif(init):
        h_mark = i_
    elif(not init):
        hist_marker += 1
    return [h_mark, enabled, weight, bias]

def new_genome(nodes = [], connects = []):
    nodes_ = nodes
    connects_ = connects
    fitness = 0
    global gene_key
    key = gene_key
    gene_key += 1
    return [key, fitness, nodes_, connects_]
    
def add_connect(nodes = [], connects = []):
    num_ns = len(nodes)
    num_cs = len(connects)
    new_c = False
    while(not new_c):
        a = random.randint(0, num_ns)
        b = random.randint(1 + num_inputs, num_ns)
        if(b == 2 + num_inputs):
            b = 0
        while(a == b):
            b = random.randrange(num_ns)
            if(b == 5):
                b = 0
        new_c = True
        for row in connects:
            if((row[4] == a) and (row[5] == b)):
                new_c = False
    connects.append(new_connect(a, b))
    return [nodes, connects]

def add_node(nodes = [], connects = []):
    num_cs = len(connects)
    h, e, w, b = new_node()
    nodes.append([h, e, w, b])
    cs_pick = random.randint(0, num_cs)
    i = 0
    for row in connects:
        if(cs_pick == i):
            row[1] = 0#this is not disabling properly
            connects.append(new_connect(row[4], h))
            c_h, c_e, c_w, c_b, c_i, c_o = new_connect(h, row[5])
            connects.append([c_h, c_e, row[3], row[4], c_i, c_o])
        i += 1
    return [nodes, connects]

def mutate_weights(nodes = [], connects = []):#and biases
    perturbation = random.uniform(-0.01, 0.01)
    for node in nodes:
        if(random.random() < weight_change_freq):
            if(random.random() < weight_uniform_cf):
                node[2] += perturbation
            else:
                node[2] = random.random()
            if(random.random() < weight_uniform_cf):
                node[3] += perturbation
            else:
                node[3] = random.random()
    for connect in connects:
        if(random.random() < weight_change_freq):
            if(random.random() < weight_uniform_cf):
                connect[2] += perturbation
            else:
                connect[2] = random.random()
            if(random.random() < weight_uniform_cf):
                connect[3] += perturbation
            else:
                connect[3] = random.random()
    return [nodes, connects]
                
#def get_action(act_n, act_c, obs):
#    action = old_get_action(0, act_n, act_c, obs)
#    amount = old_get_action(1, act_n, act_c, obs)
#    return [action, amount]

def get_action(current_hist_marker, act_n, act_c, obs):
    is_node = False
    for row in act_n:
        if(row[0] == current_hist_marker):
            is_node = True
            if(current_hist_marker < num_inputs + 1) and (current_hist_marker > 0):
                hold = run_node(act_n[current_hist_marker][:], obs[current_hist_marker - 1])
                return hold
    out_ = 0
    for c_row in act_c:
        if((c_row[5] == current_hist_marker) and (c_row[1] == 1)):
            new_c = []
            for n_row in act_c:
                if(c_row != n_row):
                    new_c.append(n_row)
            hold = get_action(c_row[4], act_n, new_c, obs)
            out_ = run_node(c_row, hold)
    return out_

def run_node(ns, y):
    if(ns[1] == 1):
        return relu(y * ns[2] + ns[3])
    else:
        return y

#todo: make the inheritence of enable with chance to enable/disable nodes/connections
#todo: keep a list of inovations in the current generation to ensure that all identical inovations recieve the same historical marker
def mate(a_nodes, a_connects, a_fitness, b_nodes, b_connects, b_fitness):
    global hist_marker
    nodes = []
    connects = []
    for i in range(hist_marker + 1):
        in_a = False
        in_b = False
        for a_node in a_nodes:
            if(a_node[0] == i):
                in_a = True
                for b_node in b_nodes:
                    if(b_node[0] == i):
                        in_b = True
                        if(a_fitness > b_fitness):
                            nodes.append(a_node)
                        elif(a_fitness < b_fitness):
                            nodes.append(b_node)
                        else:
                            if(random.random() < 0.5):
                                nodes.append(a_node)
                            else:
                                nodes.append(b_node)
        if((not in_a) and (not in_b)):
            for b_node in b_nodes:
                if(b_node[0] == i):
                    in_b = True
                    nodes.append(b_node)
        for a_connect in a_connects:
            if(a_connect[0] == i):
                in_a = True
                for b_connect in b_connects:
                    if(b_connect[0] == i):
                        in_b = True
                        if(a_fitness > b_fitness):
                            connects.append(a_connect)
                        elif(a_fitness < b_fitness):
                            connects.append(b_connect)
                        else:
                            if(random.random() < 0.5):
                                connects.append(a_connect)
                            else:
                                connects.append(b_connect)
        if((not in_a) and (not in_b)):
            for b_connect in b_connects:
                if(b_connect[0] == i):
                    in_b = True
                    connects.append(b_connect)
        k, f, n, c = new_genome(nodes, connects)
    return [k, f, n, c]

#first generation initialization
for _ in range(CONFIG_POPULATION_SIZE):
    nodes = []
    connects = []
    nodes.append(new_node(True, 0))
    for i in range(1, 2 + num_inputs):
        nodes.append(new_node(True, i))
    for i in range(1 + num_inputs, 2 + 2 * num_inputs):
        connects.append(new_connect(i - num_inputs, 0, True, i))
    hist_marker = i
    #for i in range(2 * num_inputs + 2, num_inputs * 3 + 2):
        #connects.append(new_connect(i - num_inputs * 2, 1, True, i))
    k, f, n, c = new_genome(nodes, connects)
    genomes = genomes.append({"keys": k,
                              "fitness": f,
                              "nodes": n,
                              "connects": c}, ignore_index=True)
print('hist_marker:', hist_marker)
print('init nodes:',nodes)
print('connects:', connects)

#speciation initialization
species_count = 1
species_rep = []
species_rep.append(random.randrange(CONFIG_POPULATION_SIZE))
species_list = []
for _ in range(CONFIG_POPULATION_SIZE):
    species_list.append(0)
species_best = []
species_best.append(0)
species_improved = []
species_improved.append(0) 
species_total = []
species_total.append(CONFIG_POPULATION_SIZE)
gens_since_improvement = []
gens_since_improvement.append(0)
best_ever = -100000
min_fit = []
min_fit.append(500)

mating = False

for g in range(generations):
    print("Working Generation: ", g)
    genomes_old = genomes.copy()
    best = -100000
    for i in range(species_count):
        species_improved[i] = 0
        species_best[i] = 0
        species_total[i] = 0
        for j in range(CONFIG_POPULATION_SIZE):
            if(species_list[j] == i):
                if(species_total[i] == 0):
                    species_rep[i] = j#should be a random member, not the first, but this does not have a large effect
                    species_total[i] += 1
    for k in range(CONFIG_POPULATION_SIZE):
        print('- agent: ', k)
        r_total = 0
        completes = 0
        best = 0
        done = False
        reward = 0
        env.reset(g)
        while (not done):
            if not env.should_get_action():
                env.sub_step()
            else:
                observation = env.get_current_observation()
                action = trinary_activation(get_action(0, genomes.loc[k, "nodes"], genomes.loc[k, "connects"], observation))
                env.step(action)
            done, reward = env.is_done()
        print("  Nodes: ", genomes.loc[k, "nodes"], "\nConnects: ", genomes.loc[k, "connects"])
        print('  reward = ', reward)
        genomes.loc[k, "fitness"] = reward
        if(reward > species_best[species_list[k]]):
            species_best[species_list[k]] = reward
            gens_since_improvement[species_list[k]] = 0
            species_improved[species_list[k]] = 1
        if(reward > best):
            best = reward
            if(best > best_ever):
                best_ever = best
                print("*" * 40)
                print("Best Yet Gen: ", g," Organism: ", k, " Gene Key: ", genomes.loc[k, "keys"], " Fitness: ", best_ever, "\nNodes: ", genomes.loc[k, "nodes"], "\nConnects: ", genomes.loc[k, "connects"])
                print("*" * 40)
    
    #remove the lowest performing members of each species
    for i in range(species_count):
        min_fit[i] = 500
        gens_since_improvement[i] += not(species_improved[i])
    for k in range(CONFIG_POPULATION_SIZE):
        if(genomes.loc[k, "fitness"] < min_fit[species_list[k]]):
            min_fit[species_list[k]] = genomes.loc[k, "fitness"]
    for k in range(CONFIG_POPULATION_SIZE):
        if(genomes.loc[k, "fitness"] == min_fit[species_list[k]]):
            genomes.loc[k] = genomes.loc[species_rep[species_list[k]]]
    for k in range(CONFIG_POPULATION_SIZE):
        chosen_mate = random.randint(0, CONFIG_POPULATION_SIZE - 1)
        while(chosen_mate == k):
            chosen_mate = random.randint(0, CONFIG_POPULATION_SIZE - 1)            
        if(gens_since_improvement[species_list[k]] < fitness_no_improve_limit):
            if((genomes_old.loc[k, "fitness"] < species_best[species_list[k]]) or (species_total[species_list[k]] <= 5)):
                n_, c_ = mutate_weights(genomes.loc[k, "nodes"], genomes.loc[k, "connects"])
                genomes.at[k, "nodes"] = n_
                genomes.at[k, "connects"] = c_
                if(random.random() < 0.75):
                    if(random.random() > 1 - new_node_prob):
                        genomes.at[k, "nodes"], genomes.at[k, "connects"] = add_node(genomes.loc[k, "nodes"], genomes.loc[k, "connects"])
                    if(random.random() > 1 - new_link_prob):
                        genomes.at[k, "nodes"], genomes.at[k, "connects"] = add_connect(genomes.loc[k, "nodes"], genomes.loc[k, "connects"])                        
                    if(random.random() < 0.999):
                        while((species_list[k] != species_list[chosen_mate]) or (chosen_mate == k)):
                            chosen_mate = random.randint(0, CONFIG_POPULATION_SIZE - 1)
                    genomes.at[k, "keys"], genomes.at[k, "fitness"], genomes.at[k, "nodes"], genomes.at[k, "connects"] = mate(genomes.loc[k, "nodes"], genomes.loc[k, "connects"], genomes.loc[k, "fitness"], genomes_old.loc[chosen_mate, "nodes"], genomes_old.loc[chosen_mate, "connects"], genomes_old.loc[chosen_mate, "fitness"])
                    if(not mating):
                        print("Started mating. . .")
                        mating = True
                    compatability_check = False
                    for i in range(species_count):
                        if(compatability(genomes.loc[k, "nodes"], genomes.loc[k, "connects"], genomes.loc[species_rep[species_list[k]], "nodes"], genomes.loc[species_rep[species_list[k]], "connects"])):
                            compatability_check = True
                        if(not(compatability_check)):
                            species_count += 1
                            gens_since_1
                            improvement.append(0)
                            species_best.append(0)
                            species_improved.append(0)
                            species_rep[species_count - 1] = k
                            min_fit.append(500)



hist_marker: 41
init nodes: [[0, 1, 1, 1], [1, 1, 1, 1], [2, 1, 1, 1], [3, 1, 1, 1], [4, 1, 1, 1], [5, 1, 1, 1], [6, 1, 1, 1], [7, 1, 1, 1], [8, 1, 1, 1], [9, 1, 1, 1], [10, 1, 1, 1], [11, 1, 1, 1], [12, 1, 1, 1], [13, 1, 1, 1], [14, 1, 1, 1], [15, 1, 1, 1], [16, 1, 1, 1], [17, 1, 1, 1], [18, 1, 1, 1], [19, 1, 1, 1], [20, 1, 1, 1], [21, 1, 1, 1]]
connects: [[21, 1, 1, 1, 1, 0], [22, 1, 1, 1, 2, 0], [23, 1, 1, 1, 3, 0], [24, 1, 1, 1, 4, 0], [25, 1, 1, 1, 5, 0], [26, 1, 1, 1, 6, 0], [27, 1, 1, 1, 7, 0], [28, 1, 1, 1, 8, 0], [29, 1, 1, 1, 9, 0], [30, 1, 1, 1, 10, 0], [31, 1, 1, 1, 11, 0], [32, 1, 1, 1, 12, 0], [33, 1, 1, 1, 13, 0], [34, 1, 1, 1, 14, 0], [35, 1, 1, 1, 15, 0], [36, 1, 1, 1, 16, 0], [37, 1, 1, 1, 17, 0], [38, 1, 1, 1, 18, 0], [39, 1, 1, 1, 19, 0], [40, 1, 1, 1, 20, 0], [41, 1, 1, 1, 21, 0]]
Working Generation:  0
- agent:  0
  Nodes:  [[0, 1, 1, 1], [1, 1, 1, 1], [2, 1, 1, 1], [3, 1, 1, 1], [4, 1, 1, 1], [5, 1, 1, 1], [6, 1, 1, 1], [7, 1, 1, 1], [8, 1, 1, 1], [9, 1, 1, 1], [